# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [3]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [19]:
# your code here
orders = pd.read_csv("Orders.csv")
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


In [20]:
orders.columns= orders.columns.str.lower()

---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [21]:
# your code here
orders1 = orders.groupby(['customerid']).agg({'amount_spent':sum})
orders1

,amount_spent
customerid,
12346,77183.60
12347,4310.00
12348,1797.24
12349,1757.55
12350,334.40
...,...
18280,180.60
18281,80.82
18282,178.05


In [27]:
labels = ["Peanuts", "Middle-Class", "Preferred", "VIP"]
orders1['rating'] = pd.qcut(orders1['amount_spent'], 4,labels = labels)
orders1['rating'].value_counts()

Peanuts         1085
Middle-Class    1085
VIP             1085
Preferred       1084
Name: rating, dtype: int64

In [28]:
orders1

,amount_spent,rating
customerid,,
12346,77183.60,VIP
12347,4310.00,VIP
12348,1797.24,VIP
12349,1757.55,VIP
12350,334.40,Middle-Class
...,...,...
18280,180.60,Peanuts
18281,80.82,Peanuts
18282,178.05,Peanuts


In [54]:
orders_r = pd.merge(left = orders,
                                 right = orders1,
                                 how = 'left', 
                                 left_on = "customerid", 
                                 right_on= "customerid")
display(orders_r.shape)
orders_r.head()

(397924, 17)

,unnamed: 0,invoiceno,stockcode,year,month,day,hour,description,quantity,invoicedate,unitprice,customerid,country,amount_spent_x,rating_x,amount_spent_y,rating_y
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,NaN,5391.21,VIP
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,NaN,5391.21,VIP
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00,NaN,5391.21,VIP
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,NaN,5391.21,VIP
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,NaN,5391.21,VIP


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [58]:
# your code here
vips = orders_r.groupby(['country', 'customerid']).agg({'rating_y':max})
result = []
for x in vips['rating_y']:
    if  x == 'VIP':
        result.append(1)
    else:
        result.append(0)
vips['vip'] = result
vips = vips.groupby(['country']).agg({'vip':sum})
vips = vips['vip'].idxmax()
vips

'United Kingdom'

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [59]:
# your code here
vip_pref = orders_r.groupby(['country', 'customerid']).agg({'rating_y':max})
result1 = []
for x in vip_pref['rating_y']:
    if  x == 'VIP':
        result1.append(1)
    elif x ==  'Preferred':
        result1.append(1)
    else:
        result1.append(0)
vip_pref['vip'] = result1
vip_pref = vip_pref.groupby(['country']).agg({'vip':sum})
vip_pref = vip_pref['vip'].idxmax()
vip_pref

'United Kingdom'